In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as f
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader,Dataset,random_split
import PIL
import random
if (torch.cuda.is_available()):
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [18]:
class Network(nn.Module):
    def __init__(self):   
        super().__init__()
        self.l1 = nn.Conv2d(3,32,3,padding = 1)
        self.n1 = nn.BatchNorm2d(32)
        self.l2 = nn.Conv2d(32,32,3,padding = 1)
        self.n2 = nn.BatchNorm2d(32)
        self.l3 = nn.Conv2d(32,64,3,padding = 1)
        self.n3 = nn.BatchNorm2d(64)
        self.l4 = nn.Conv2d(64,64,3,padding = 1)
        self.n4 = nn.BatchNorm2d(64)
        self.l5 = nn.Conv2d(64,128,3,padding = 1)
        self.n5 = nn.BatchNorm2d(128)
        self.l6 = nn.Conv2d(128,128,3,padding = 1)
        self.n6 = nn.BatchNorm2d(128)
        self.l7 = nn.Conv2d(128,128,3,padding = 1)
        self.n7 = nn.BatchNorm2d(128)
        self.l8 = nn.Conv2d(128,256,3,padding = 1)
        self.n8 = nn.BatchNorm2d(256)
        self.l9 = nn.Conv2d(256,256,3,padding = 1)
        self.n9 = nn.BatchNorm2d(256)
        self.l10 = nn.Linear(9216,1024)
        self.l11 = nn.Linear(1024,1024)
        self.c = nn.Sequential(
            nn.Linear(9216,10),
            nn.Softmax(dim = 1)
        )
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p = 0.3)
        self.drop2d = nn.Dropout2d(p = 0.3)
        self.softmax = nn.Softmax()
        self.pool = nn.MaxPool2d(2,stride = 2)
    def forward(self,xt,flag = False):
      x = xt
      convert = torch.zeros((x.shape[0],32-x.shape[1],x.shape[2],x.shape[3]))
      convert = convert.to(device)
      xt = torch.cat([xt,convert],dim = 1)
      x = self.l1(x)
      x = self.n1(x)
      x = self.relu(x)
      x = self.drop2d(x)
      x = self.l2(x)
      x = x+xt
      x = self.n2(x)
      x = self.relu(x)
      x = self.pool(x)
      xt = x
      x = self.l3(x)
      x = self.n3(x)
      x = self.relu(x)
      x = self.drop2d(x)
      x = self.l4(x)
      convert = torch.zeros(xt.shape,device=device)
      xt = torch.cat([xt,convert],dim = 1)
      x = x + torch.reshape(xt,x.shape)
      x = self.n4(x)
      x = self.relu(x)
      x = self.pool(x)
      xt = x
      x = self.l5(x)
      x = self.n5(x)
      x = self.relu(x)
      x = self.drop2d(x)
      x = self.l6(x)
      convert = torch.zeros(xt.shape,device=device)
      xt = torch.cat([xt,convert],dim = 1)
      x = x + torch.reshape(xt,x.shape)
      x = self.n6(x)
      x = self.relu(x)
      x = self.drop2d(x)
      x = self.pool(x)
      xt = x
      x = self.l7(x)
      x = self.n7(x)
      x = self.relu(x)
      x = self.drop2d(x)
      x = self.l8(x)
      convert = torch.zeros(xt.shape,device=device)
      xt = torch.cat([xt,convert],dim = 1)
      x = x + torch.reshape(xt,x.shape)
      x = self.n8(x)
      x = self.relu(x)
      x = self.pool(x)
      x = self.l9(x)
      x = self.n9(x)
      x = self.relu(x)
      x = torch.reshape(x,(x.shape[0],-1,1,1))
      x = torch.squeeze(x,dim = 2)
      x = torch.squeeze(x,dim = 2)
      if flag:
        x = self.l10(x)
        x = self.relu(x)
        x = self.l11(x)
      else:
        x = self.c(x)
      return x
model = Network()
model = model.to(device)

In [3]:
class Unsoupdata(Dataset):
  def __init__(self):
    super().__init__()
    transform = transforms.ToTensor()
    self.t = transforms.Compose([
                            transforms.RandomVerticalFlip(0.5),
                            transforms.RandomHorizontalFlip(0.5),
                            transforms.RandomApply([transforms.ColorJitter()],p=0.5)
    ])
    self.data = torchvision.datasets.STL10(root = '/',split = 'unlabeled',download = True,transform = transform)
    self.n = len(self.data)
  def __len__(self):
    return len(self.data)
  def __getitem__(self,i):
    decide = random.randint(0,self.n-1)
    while (decide==i):    
      decide = random.randint(0,self.n-1)
    return self.data[i][0],self.t(self.data[i][0]),self.data[decide][0],self.t(self.data[decide][0])

class Soupdata(Dataset):
  def __init__(self):
    super().__init__()
    transform = transforms.ToTensor()
    self.data = torchvision.datasets.STL10(root = '/',split = 'train',download = True,transform = transform)
    self.n = len(self.data)
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self,i):
    return self.data[i]

In [4]:
batch_size = 128
unsoup = Unsoupdata()
unsouploader = DataLoader(dataset=unsoup,batch_size = 128,shuffle = True)
soup =  Soupdata()
train_data,test_val = random_split(soup,[len(soup)-500,500])
test_data = torchvision.datasets.STL10(root = '/',split = 'test',download = True,transform = transforms.ToTensor())
souptrain = DataLoader(dataset=train_data,batch_size = batch_size,shuffle = True)
soupval = DataLoader(dataset=test_val,batch_size = len(test_val),shuffle = True)
souptest = DataLoader(dataset=test_data,batch_size = len(test_data),shuffle = True)

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting /stl10_binary.tar.gz to /
Files already downloaded and verified
Files already downloaded and verified


In [19]:
coste = nn.TripletMarginLoss()
costc = nn.CrossEntropyLoss()
sim = []
def loss(y1,y2,y3,y4,t = 1,eps = 1e-8):
  siml = nn.CosineSimilarity(dim = 1)
  positive = siml(y1,y2)*siml(y3,y4)
  negative = siml(y1,y3)+siml(y1,y4)+siml(y2,y3)+siml(y2,y4)
  negative = negative**2
  positive = torch.exp(positive)/t
  negative = torch.exp(negative)/t
  los = -1*torch.log(positive/negative)
  los = los.mean()
  return los

In [20]:
optimizerc = torch.optim.Adam(model.c.parameters(),lr = 0.001)
optimizere = torch.optim.Adam(model.parameters(),lr = 0.001)
def test(data):
    c = 0
    s = 0
    for i,(x,y) in enumerate(data):
        with torch.no_grad():
            x =x.to(device)
            y = y.to(device,dtype = torch.int64)
            yt = model(x,False)
            yt = torch.argmax(yt, dim= 1)
            c = (y == yt).sum()
            s = y.shape[0]
        break
    return (100*c/s).item()

In [21]:
epochs = 10
for j in range(epochs):
  for i,(x1,x2,x3,x4) in enumerate(unsouploader) :
    x1 = x1.to(device)
    x2 = x2.to(device)
    x3 = x3.to(device)
    x4 = x4.to(device)
    y1 = model(x1,True)
    y2 = model(x2,True)
    y3 = model(x3,True)
    y4 = model(x4,True)
    losse = loss(y1,y2,y3,y4)          
    optimizere.zero_grad()
    losse.backward()
    optimizere.step()
    print(f'epoch {j+1} step {i} loss {losse}')


Streaming output truncated to the last 5000 lines.
epoch 1 step 389 loss 0.46476173400878906
epoch 1 step 390 loss 0.612053394317627
epoch 1 step 391 loss 0.6649968028068542
epoch 1 step 392 loss 0.447976678609848
epoch 1 step 393 loss 0.6037282943725586
epoch 1 step 394 loss 0.9113625288009644
epoch 1 step 395 loss 0.5033005475997925
epoch 1 step 396 loss 0.5262120962142944
epoch 1 step 397 loss 0.5894442796707153
epoch 1 step 398 loss 0.5562207698822021
epoch 1 step 399 loss 0.48075544834136963
epoch 1 step 400 loss 0.5172708630561829
epoch 1 step 401 loss 0.5605555772781372
epoch 1 step 402 loss 0.4943723678588867
epoch 1 step 403 loss 0.486528217792511
epoch 1 step 404 loss 0.6501578092575073
epoch 1 step 405 loss 0.3585273027420044
epoch 1 step 406 loss 0.5527539849281311
epoch 1 step 407 loss 0.5702358484268188
epoch 1 step 408 loss 0.4440802037715912
epoch 1 step 409 loss 0.5926536917686462
epoch 1 step 410 loss 0.6106722950935364
epoch 1 step 411 loss 0.5288556814193726
epoch 1

KeyboardInterrupt: ignored

In [22]:
epochs = 40
for j in range(epochs):
   for i,(xt,yt) in enumerate(souptrain) :
    xt = xt.to(device)
    #xt = torch.reshape(xt,[xt.shape[0],-1,1,1])
    yt = yt.to(device)
    #xt = torch.squeeze(xt,dim = 2)
    #xt = torch.squeeze(xt,dim = 2)
    y_pred = model(xt,False)
    y_pred = torch.squeeze(y_pred)
    lossc = costc(y_pred,yt)          
    optimizerc.zero_grad()
    lossc.backward()
    optimizerc.step()
    if (i%20==0):
      acc = test(soupval)
      print(f'epoch {j+1} step {i} loss {lossc} test_accuracy {acc} train_accuracy {test(souptrain)}')
    else:
      print(f'epoch {j+1} step {i} loss {lossc}')

epoch 1 step 0 loss 2.308993101119995 test_accuracy 10.800000190734863 train_accuracy 8.59375
epoch 1 step 1 loss 2.305510997772217
epoch 1 step 2 loss 2.252635955810547
epoch 1 step 3 loss 2.3051791191101074
epoch 1 step 4 loss 2.3354108333587646
epoch 1 step 5 loss 2.2735636234283447
epoch 1 step 6 loss 2.3025429248809814
epoch 1 step 7 loss 2.2513794898986816
epoch 1 step 8 loss 2.227808952331543
epoch 1 step 9 loss 2.310154676437378
epoch 1 step 10 loss 2.2273454666137695
epoch 1 step 11 loss 2.2776827812194824
epoch 1 step 12 loss 2.2411158084869385
epoch 1 step 13 loss 2.2005467414855957
epoch 1 step 14 loss 2.2553844451904297
epoch 1 step 15 loss 2.270887613296509
epoch 1 step 16 loss 2.252403497695923
epoch 1 step 17 loss 2.2104296684265137
epoch 1 step 18 loss 2.2278342247009277
epoch 1 step 19 loss 2.2626090049743652
epoch 1 step 20 loss 2.2274460792541504 test_accuracy 20.0 train_accuracy 20.3125
epoch 1 step 21 loss 2.211432695388794
epoch 1 step 22 loss 2.296591281890869
e

In [23]:
souptest = DataLoader(dataset=test_data,batch_size = 64,shuffle = True)
def teste(data):
    c = 0
    s = 0
    for x,y in (data):
        with torch.no_grad():
            x =x.to(device)
            y = y.to(device)
            yt = model(x)
            yt = torch.squeeze(yt)
            yt = torch.argmax(yt, dim= 1)
            c += (y == yt).sum()
            s += y.shape[0]
    return (100*c/s).item()

teste(souptest)

26.400001525878906